# Pre-Processing : Data Preparation

---
Data preparation and cleaning to suit the identified problem

### Essential Libraries

> NumPy : Library for Numeric Computations in Python  
> Pandas : Library for Data Acquisition and Preparation  
> Matplotlib : Low-level library for Data Visualization  
> Seaborn : Higher-level library for Data Visualization  

In [1]:
# Basic Libraries
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
import pickle
sb.set() # set the default Seaborn style for graphics

---

## Setup : Import the Dataset

Dataset from Kaggle : `Company Reviews`
Source: https://www.kaggle.com/datasets/vaghefi/company-reviews?datasetId=848918&sortBy=dateRun&tab=collaboration

The dataset is `company_reviews.csv`; hence we use the `read_csv` function from Pandas.  

In [2]:
rawData = pd.read_csv('../dataset/company_reviews.csv')
print("The raw dataset dimension : ", rawData.shape) 
rawData.head()

The raw dataset dimension :  (17050, 20)


,name,rating,reviews,description,happiness,ceo_approval,ceo_count,ratings,locations,roles,salary,interview_experience,interview_difficulty,interview_duration,interview_count,headquarters,employees,industry,revenue,website
0,Sitel,NaN,NaN,"Sitel Group’s 75,000 people across the globe c...","{'Work Happiness Score': '55', 'Achievement': ...",70%,"CEO Approval is based on 4,612 ratings","{'Work/Life Balance': '3.4', 'Compensation/Ben...","{'Paradise, NV': '5.0', 'Pioneer, OH': '4.7', ...","{'Tier 1 Agent': '5.0', 'Director of Operation...",{'Customer Service Representative': '$14.48 pe...,Favorable,Easy,About a day or two,Based on 645 interviews,"600 Brickell Ave Miami, FL 33131 Vereinigte St...","10,000+",Telecommunications,$1B to $5B (USD),Twitter\nFacebook\nSitel website
1,Meadowbrook Rehabilitation,3.7,21 reviews,You'll work with the most experienced and loya...,{},NaN,NaN,"{'Work/Life Balance': '4.1', 'Compensation/Ben...",{},{},{},Favorable,Easy,NaN,Based on 5 interviews,Illinois,NaN,Healthcare,NaN,Meadowbrook Rehabilitation website
2,Intermountain,4.0,23 reviews,Why Intermountain?\n\nWe Bring Hope\n\nWith ou...,{},88%,CEO Approval is based on 17 ratings,"{'Work/Life Balance': '3.5', 'Compensation/Ben...",{},{},{'Mental Health Technician': '$13.16 per hour'...,Favorable,Medium,About a day or two,Based on 8 interviews,"Headquarters: 3240 Dredge Dr. Helena, MT 59602",201 to 500,Healthcare,$5M to $25M (USD),Twitter\nFacebook\nIntermountain website
3,Smith & Nephew,NaN,NaN,It's more than business at Smith+Nephew - it's...,"{'Work Happiness Score': '65', 'Purpose': '71'...",76%,CEO Approval is based on 374 ratings,"{'Work/Life Balance': '3.5', 'Compensation/Ben...","{'Largo, FL': '4.5', 'Chicago, IL': '4.3', 'Sa...","{'Packaging Technician': '5.0', 'Senior Associ...","{'Packager': '$30,006 per year', 'Finisher': '...",Favorable,Medium,About a week,Based on 116 interviews,"Building 5, Croxley Park, Hatters Lane, Watfor...","10,000+",Healthcare,$1B to $5B (USD),Twitter\nFacebook\nSmith & Nephew website
4,Reverse Mortgage Funding,4.1,19 reviews,Reverse Mortgage Funding LLC is committed to e...,{},NaN,NaN,"{'Work/Life Balance': '4.2', 'Compensation/Ben...",{},{},{},Favorable,NaN,NaN,Based on 5 interviews,Bloomfield,11 to 50,Financial Services,$5M to $25M (USD),Reverse Mortgage Funding website


---

## Clean up data

1. Drop rows with missing data
2. Drop unnecessary columns
3. Drop data lack of credibility 
    

In [3]:
data_clean1 = rawData.replace(to_replace = '{}', value=np.nan)
data_clean1.isnull().sum()

name                      338
rating                   1434
reviews                  1556
description                 1
happiness               12463
ceo_approval             5828
ceo_count                5828
ratings                  1281
locations                9922
roles                    9922
salary                   5427
interview_experience     5551
interview_difficulty     5588
interview_duration       6728
interview_count          5551
headquarters             1820
employees                2027
industry                 1846
revenue                  6873
website                   921
dtype: int64

----
### Step 1: Drop Rows (selected) with Missing Values
We will delete the rows without rating (predictor)<br>
Then, we drop the rows without the values for potential features to be used in prediction
- ratings (including specific ratings of multiple perspectives, i.e. work-life balance)
- reviews (drop rows without reviews to ensure data credibility)
- industry (the ratings will be predicted with regard to the industry)
- employees (information is defective)
- revenue (information is defective)

In [4]:
data_clean1 = data_clean1.dropna(subset = ['rating', 'ratings', 'reviews', 'industry', 'employees','revenue'])
print("The dataset dimension after dropping rows (selectd) with missing values : ", data_clean1.shape)


The dataset dimension after dropping rows (selectd) with missing values :  (8716, 20)


----
### Step 2: Drop Unnecessary Columns
We will drop columns that not to be used for prediction<br>
- company name
- description
- locations
- interview_experience
- interview_difficulty
- interview_duration
- interview_count
- headquarters
- website

In [5]:
data_clean2 = data_clean1.drop(['name', 'description','locations','interview_experience','interview_difficulty', 'interview_duration',
       'interview_count','headquarters','website',], axis=1)
print("The dataset dimension after dropping unnecessary cloumns : ", data_clean2.shape) 


The dataset dimension after dropping unnecessary cloumns :  (8716, 11)


----
### Step 3: Drop Data Lack of Credibility
We will drop company record that only has a few reviews records to ensure the credibility
> Inspecting the reviews columns, the median reviews received is `60`, and the 1st quartile is `20`, we will delete the records in 1st quartile as the ratings are not reflected by enough people.

In [6]:
data_clean2['reviews'] = data_clean2['reviews'].replace([' reviews', 'reviews'], '', regex=True)
data_clean2['reviews'] = data_clean2['reviews'].replace({'K': '*1e3'}, regex=True).map(pd.eval).astype(int)
data_clean2['reviews'] = pd.to_numeric(data_clean2['reviews'])
data_clean2['reviews'].describe()

count      8716.000000
mean        725.487494
std        4007.680596
min           2.000000
25%          34.000000
50%         102.000000
75%         366.000000
max      215500.000000
Name: reviews, dtype: float64

In [7]:
data_clean3 = data_clean2.drop(data_clean2[data_clean2.reviews <= 34].index)
print("The dataset dimension after dropping data lack of credibility : ", data_clean3.shape) 


The dataset dimension after dropping data lack of credibility :  (6476, 11)


---

## Prepare Data
#### Step 1: Split data in column `ratings` to individual columns
- Work/Life Balance
- Compensation/Benefits
- Job Security/Advancement
- Management
- Culture

In [8]:
df = data_clean3
df["ratings"] = df["ratings"].apply(lambda x : dict(eval(x)) )
df1 = df["ratings"].apply(pd.Series)

data_prepare1 = pd.concat([df, df1], axis=1).drop('ratings', axis=1)
data_prepare1[['Work/Life Balance', 'Compensation/Benefits', 'Job Security/Advancement', 'Management', 'Culture']] = data_prepare1[['Work/Life Balance', 'Compensation/Benefits', 'Job Security/Advancement', 'Management', 'Culture']].apply(pd.to_numeric)
data_prepare1.head()

,rating,reviews,happiness,ceo_approval,ceo_count,roles,salary,employees,industry,revenue,Work/Life Balance,Compensation/Benefits,Job Security/Advancement,Management,Culture
5,3.4,437,NaN,57%,CEO Approval is based on 199 ratings,"{'Workday HCM Consultant': '5.0', 'Software Te...",{'Customer Service Representative': '$29.97 pe...,"5,001 to 10,000",Manufacturing,$1B to $5B (USD),3.4,3.5,2.8,2.9,3.2
7,3.5,367,NaN,78%,CEO Approval is based on 167 ratings,"{'Claims Examiner': '4.7', 'Office Assistant':...","{'Insurance Agent': '$48,284 per year', 'Agent...","1,001 to 5,000",Insurance,$1B to $5B (USD),3.7,3.3,3.2,3.1,3.3
9,4.2,803,"{'Work Happiness Score': '76', 'Appreciation':...",83%,CEO Approval is based on 275 ratings,"{'Mentor': '5.0', 'Intern': '5.0', 'Security C...","{'Graduate Researcher': '$26,598 per year', 'R...","5,001 to 10,000",Education,$1B to $5B (USD),4.0,3.7,3.6,3.7,4.0
16,4.1,501,"{'Work Happiness Score': '70', 'Learning': '81...",89%,CEO Approval is based on 181 ratings,NaN,"{'Program Coordinator': '$48,016 per year', 'R...","5,001 to 10,000",Education,$1B to $5B (USD),4.1,3.8,3.7,3.7,3.9
17,4.0,183,NaN,76%,CEO Approval is based on 84 ratings,"{'Laboratory Assistant': '5.0', 'Senior Resear...",NaN,"5,001 to 10,000",Manufacturing,more than $10B (USD),4.0,3.7,3.4,3.6,3.9


#### Step 2: Deal with Datatype of Each Column
- ceo_count (int64)
- ceo_approval (int64)
- employees (categorical)
- revenue (categorical)

In [9]:
data_prepare1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6476 entries, 5 to 17048
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   rating                    6476 non-null   float64
 1   reviews                   6476 non-null   int64  
 2   happiness                 3117 non-null   object 
 3   ceo_approval              6443 non-null   object 
 4   ceo_count                 6443 non-null   object 
 5   roles                     4892 non-null   object 
 6   salary                    5662 non-null   object 
 7   employees                 6476 non-null   object 
 8   industry                  6476 non-null   object 
 9   revenue                   6476 non-null   object 
 10  Work/Life Balance         6476 non-null   float64
 11  Compensation/Benefits     6476 non-null   float64
 12  Job Security/Advancement  6476 non-null   float64
 13  Management                6476 non-null   float64
 14  Culture

---
##### Handle `ceo_count`
- substract numeric data as integer
- handle NaN data 

In [10]:
data_prepare1['ceo_count'] = data_prepare1['ceo_count'].replace([np.nan], '0', regex=True)
data_prepare1['ceo_count'] = data_prepare1['ceo_count'].replace(['CEO Approval is based on ', ' ratings', ','], '', regex=True)
data_prepare1['ceo_count'] = pd.to_numeric(data_prepare1['ceo_count'])
data_prepare1['ceo_count'] 

5        199
7        167
9        275
16       181
17        84
        ... 
17035     63
17041    121
17042    485
17044    131
17048     33
Name: ceo_count, Length: 6476, dtype: int64

---
##### Handle `ceo_approval`
- convert percentage to int64
- handle NaN data by replacing it with mean


In [11]:
data_prepare1['ceo_approval'] = data_prepare1['ceo_approval'].replace([np.nan], '0', regex=True)
data_prepare1['ceo_approval'] = data_prepare1['ceo_approval'].replace(['%'], '', regex=True)
data_prepare1['ceo_approval'] = pd.to_numeric(data_prepare1['ceo_approval'])

ceo_approval_mean = sum(data_prepare1['ceo_approval'] * data_prepare1['ceo_count']) / sum(data_prepare1['ceo_count'])
print("The mean of ceo approval is", ceo_approval_mean)

The mean of ceo approval is 74.25473456515891


In [12]:
data_prepare1['ceo_approval'] = data_prepare1['ceo_approval'].replace(['0'], int(ceo_approval_mean), regex=False)
data_prepare1['ceo_approval'] 

5        57
7        78
9        83
16       89
17       76
         ..
17035    65
17041    44
17042    87
17044    58
17048    88
Name: ceo_approval, Length: 6476, dtype: int64

---
##### Handle `employees`
- relabel the categorical data
| Label  |Description       | Count       |    
| :---                | :---: | :----: |       
| 1   |  1             |    6    |            
| 2   |  2 to 10       |   66    |   
| 3   |  11 to 50      |   101   |       
| 4   |  51 to 200     |   365   |       
| 5   | 201 to 500     |   755   |       
| 6   | 501 to 1,000   |   954   |       
| 7   |1,001 to 5,000  |   2135  |       
| 8   |5,001 to 10,000 |   734   | 
| 9   |  10,000+       |   1360  | 

In [13]:
pd.unique(data_prepare1['employees'])

array(['5,001 to 10,000', '1,001 to 5,000', '201 to 500', '10,000+',
       '501 to 1,000', '51 to 200', '11 to 50', '2 to 10', '1'],
      dtype=object)

In [14]:
employees_mapper = {"1":1, 
                    "2 to 10":2, 
                    "11 to 50":3, 
                    "51 to 200":4,
                    "201 to 500":5,
                    "501 to 1,000":6,
                    "1,001 to 5,000":7, 
                    "5,001 to 10,000":8, 
                    "10,000+":9}
data_prepare1['employees']=data_prepare1['employees'].replace(employees_mapper)
data_prepare1['employees'].value_counts().sort_index()


1       6
2      66
3     101
4     365
5     755
6     954
7    2135
8     734
9    1360
Name: employees, dtype: int64

---
##### Handle `revenue`
- relabel the categorical data

| Label  |Description       | Count       |    
| :---                | :------: | :--------: |    
| 1   |  less than 1M (USD)  |    233 |            
| 2   |  1 to 5M (USD)      |    298 |   
| 3   |  5M to 25M (USD)    |    667 |       
| 4   |  25M to 100M (USD) |    1127|       
| 5   |  100M to 500M (USD) |   1541 |       
| 6   |  500M to 1B (USD)   |   626  |       
| 7   |  1B to 5B (USD)     |   1169 |       
| 8   |  5B to 10B (USD)    |   306  | 
| 9   |  more than 10B (USD) |   509  | 

In [15]:
pd.unique(data_prepare1['revenue'])

array(['$1B to $5B (USD)', 'more than $10B (USD)', '$100M to $500M (USD)',
       '$500M to $1B (USD)', '$5B to $10B (USD)', '$25M to $100M (USD)',
       '$5M to $25M (USD)', '$1M to $5M (USD)', 'less than $1M (USD)'],
      dtype=object)

In [16]:
revenue_mapper = {"less than $1M (USD)":1, 
                    "$1M to $5M (USD)":2, 
                    "$5M to $25M (USD)":3, 
                    "$25M to $100M (USD)":4,
                    "$100M to $500M (USD)":5,
                    "$500M to $1B (USD)":6,
                    "$1B to $5B (USD)":7, 
                    "$5B to $10B (USD)":8, 
                    "more than $10B (USD)":9}

data_prepare1['revenue']=data_prepare1['revenue'].replace(revenue_mapper)
data_prepare1['revenue'].value_counts().sort_index()


1     233
2     298
3     667
4    1127
5    1541
6     626
7    1169
8     306
9     509
Name: revenue, dtype: int64

#### Step 3: Deal with Industry Data
Further Categorise the Industry data

In [17]:
data_prepare1['industry'].unique()

array(['Manufacturing', 'Insurance', 'Education',
       'Transportation & Logistics', 'Energy, Mining & Utilities',
       'Consulting and Business Services', 'Healthcare',
       'Financial Services', 'Restaurants, Travel and Leisure',
       'Information Technology', 'Telecommunications',
       'Construction & Facilities Services', 'Retail & Wholesale',
       'Nonprofit & NGO', 'Food and Beverages', 'Aerospace & Defense',
       'Real Estate', 'Retail', 'Consumer Goods and Services',
       'Human Resources & Staffing', 'Pharmaceutical & Biotechnology',
       'Automotive', 'Computers and Electronics', 'Media & Communication',
       'Government & Public Administration', 'Agriculture and Extraction',
       'Hotels & Travel Accommodation', 'Personal Consumer Services',
       'Restaurants & Food Service', 'Management & Consulting',
       'Manufacturing\nConstruction & Facilities Services', 'Agriculture',
       'Arts, Entertainment & Recreation', 'Legal',
       'Energy, Mining &

In [18]:
Financials = ['Financial Services', 'Insurance', 'Consulting and Business Services', 'Human Resources & Staffing', 
              'Management & Consulting', 'Insurance\nHealthcare', 'Insurance\nFinancial Services']
Leisure = ['Restaurants, Travel and Leisure', 'Hotels & Travel Accommodation', 'Restaurants & Food Service', 
           'Arts, Entertainment & Recreation', 'Restaurants, Travel and Leisure\nRestaurants, Travel and Leisure']
Healthcare_NGO = ['Healthcare', 'Nonprofit & NGO', 'Healthcare\nNonprofit & NGO', 'Pharmaceutical & Biotechnology', 
                  'Health Care', 'Pharmaceuticals']
Telecomm = ['Media & Communication', 'Telecommunications']
Govt_Services = ['Education', 'Aerospace & Defense', 'Government & Public Administration', 'Legal', 
                 'Transportation & Logistics']
Tech = ['Computers and Electronics', 'Internet and Software', 'Information Technology']
Commodities = ['Manufacturing', 'Energy, Mining & Utilities', 'Agriculture and Extraction', 'Agriculture', 
               'Agriculture and Extraction\nAgriculture and Extraction', 'Manufacturing\nEnergy, Mining & Utilities',
              'Manufacturing\nConstruction & Facilities Services', 'Energy, Mining & Utilities\nFinancial Services',
              'Transport and Freight']
Real_Estate = ['Construction & Facilities Services', 'Real Estate']
FNB = ['Restaurants & Food Service', 'Food and Beverages']
Retail = ['Retail & Wholesale', 'Retail', 'Consumer Goods and Services', 'Automotive', 'Personal Consumer Services']

In [19]:
def search(list, platform):
    for i in list:
        if i == platform:
            return True
    return False

def sort_sector(oldSector, sectorList, newSector):
    if oldSector in sectorList:
        return newSector
    else:
        return oldSector
    
data_prepare1['industry'] = data_prepare1['industry'].apply(lambda x: sort_sector(x, Financials, 'Financials'))
data_prepare1['industry'] = data_prepare1['industry'].apply(lambda x: sort_sector(x, Leisure, 'Leisure'))
data_prepare1['industry'] = data_prepare1['industry'].apply(lambda x: sort_sector(x, Healthcare_NGO, 'Healthcare_NGO'))
data_prepare1['industry'] = data_prepare1['industry'].apply(lambda x: sort_sector(x, Telecomm, 'Telecomm'))
data_prepare1['industry'] = data_prepare1['industry'].apply(lambda x: sort_sector(x, Govt_Services, 'Govt_Services'))
data_prepare1['industry'] = data_prepare1['industry'].apply(lambda x: sort_sector(x, Tech, 'Tech'))
data_prepare1['industry'] = data_prepare1['industry'].apply(lambda x: sort_sector(x, Commodities, 'Commodities'))
data_prepare1['industry'] = data_prepare1['industry'].apply(lambda x: sort_sector(x, FNB, 'FNB'))
data_prepare1['industry'] = data_prepare1['industry'].apply(lambda x: sort_sector(x, Retail, 'Retail'))
data_prepare1['industry'] = data_prepare1['industry'].apply(lambda x: sort_sector(x, Real_Estate, 'Real_Estate'))

'''
data_prepare1['industry'] = data_prepare1['industry'].replace(Financials, 'Financials', regex=True)
data_prepare1['industry'] = data_prepare1['industry'].replace(Leisure, 'Leisure', regex=True)
data_prepare1['industry'] = data_prepare1['industry'].replace(Healthcare_NGO, 'Healthcare_NGO', regex=True)
data_prepare1['industry'] = data_prepare1['industry'].replace(Telecomm, 'Telecomm', regex=True)
data_prepare1['industry'] = data_prepare1['industry'].replace(Govt_Services, 'Govt_Services', regex=True)
data_prepare1['industry'] = data_prepare1['industry'].replace(Tech, 'Tech', regex=True)
data_prepare1['industry'] = data_prepare1['industry'].replace(Commodities, 'Commodities', regex=True)
data_prepare1['industry'] = data_prepare1['industry'].replace(FNB, 'FNB', regex=True)
data_prepare1['industry'] = data_prepare1['industry'].replace(Retail, 'Retail', regex=True)
data_prepare1['industry'] = data_prepare1['industry'].replace(Real_Estate, 'Real_Estate', regex=True)
'''

data_prepare1['industry'].unique()

array(['Commodities', 'Financials', 'Govt_Services', 'Healthcare_NGO',
       'Leisure', 'Tech', 'Telecomm', 'Real_Estate', 'Retail', 'FNB'],
      dtype=object)

#### Step 4: Calculate average salary (assuming simple average)

In [20]:
def avg_salary(salaryDict):
    total = 0
    count = 0
    for value in list(salaryDict.values()):
        value = value.replace('$', '')
        value = value.replace(',', '')
        if 'year' in value:
            value = value.replace(' per year', '')
            value = float(value)
        elif 'month' in value:
            value = value.replace(' per month', '')
            value = float(value)*12    
        elif 'week' in value:
            value = value.replace(' per week', '')
            value = float(value)*52 #assume 52 weeks a year
        elif 'day' in value:
            value = value.replace(' per day', '')
            value = float(value)*5*52 #assume 5 days a week, 52 weeks a year
        elif 'hour' in value:
            value = value.replace(' per hour', '')
            value = float(value)*40*52 #assume 40 hours a week, 52 weeks a year
            
        count += 1
        total += value
    
    return total/count

data_prepare1['avg_salary'] = data_prepare1['salary'].apply(lambda x: x if x is np.nan else avg_salary(eval(x)))

data_prepare1

,rating,reviews,happiness,ceo_approval,ceo_count,roles,salary,employees,industry,revenue,Work/Life Balance,Compensation/Benefits,Job Security/Advancement,Management,Culture,avg_salary
5,3.4,437,NaN,57,199,"{'Workday HCM Consultant': '5.0', 'Software Te...",{'Customer Service Representative': '$29.97 pe...,8,Commodities,7,3.4,3.5,2.8,2.9,3.2,83730.200000
7,3.5,367,NaN,78,167,"{'Claims Examiner': '4.7', 'Office Assistant':...","{'Insurance Agent': '$48,284 per year', 'Agent...",7,Financials,7,3.7,3.3,3.2,3.1,3.3,40685.666667
9,4.2,803,"{'Work Happiness Score': '76', 'Appreciation':...",83,275,"{'Mentor': '5.0', 'Intern': '5.0', 'Security C...","{'Graduate Researcher': '$26,598 per year', 'R...",8,Govt_Services,7,4.0,3.7,3.6,3.7,4.0,36279.933333
16,4.1,501,"{'Work Happiness Score': '70', 'Learning': '81...",89,181,NaN,"{'Program Coordinator': '$48,016 per year', 'R...",8,Govt_Services,7,4.1,3.8,3.7,3.7,3.9,45855.000000
17,4.0,183,NaN,76,84,"{'Laboratory Assistant': '5.0', 'Senior Resear...",NaN,8,Commodities,9,4.0,3.7,3.4,3.6,3.9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17035,3.6,174,NaN,65,63,NaN,"{'Registered Nurse': '$55,140 per year', 'Regi...",7,Healthcare_NGO,5,3.5,3.4,3.1,3.0,3.3,53226.800000
17041,2.5,255,"{'Work Happiness Score': '46', 'Achievement': ...",44,121,"{'Vehicle Operator': '4.0', 'Inspector': '3.9'...","{'Shuttle Driver': '$12.54 per hour', 'Driver'...",7,Govt_Services,5,2.4,2.6,2.4,2.2,2.4,28336.533333
17042,3.9,1600,"{'Work Happiness Score': '72', 'Achievement': ...",87,485,"{'Office Administrator': '5.0', 'Senior Field ...","{'Logistics Analyst': '$62,419 per year', 'Sys...",8,Financials,7,3.8,3.9,3.4,3.5,3.7,83891.000000
17044,3.3,311,NaN,58,131,"{'Recruiter': '5.0', 'Office Assistant': '4.8'...","{'Food Service Associate': '$12.14 per hour', ...",8,Govt_Services,5,3.2,3.3,3.3,3.3,3.3,31330.200000


## Export processed data to excel and pickle (V1)

In [21]:
data_prepare1 = data_prepare1.reset_index(drop=True)
data_prepare1.index = np.arange(1, len(data_prepare1) + 1)
data_prepare1.to_excel("../dataset/data_prepare1.xlsx")
data_prepare1.to_pickle('../dataset/data_prepare1.pickle')

data_prepare1

,rating,reviews,happiness,ceo_approval,ceo_count,roles,salary,employees,industry,revenue,Work/Life Balance,Compensation/Benefits,Job Security/Advancement,Management,Culture,avg_salary
1,3.4,437,NaN,57,199,"{'Workday HCM Consultant': '5.0', 'Software Te...",{'Customer Service Representative': '$29.97 pe...,8,Commodities,7,3.4,3.5,2.8,2.9,3.2,83730.200000
2,3.5,367,NaN,78,167,"{'Claims Examiner': '4.7', 'Office Assistant':...","{'Insurance Agent': '$48,284 per year', 'Agent...",7,Financials,7,3.7,3.3,3.2,3.1,3.3,40685.666667
3,4.2,803,"{'Work Happiness Score': '76', 'Appreciation':...",83,275,"{'Mentor': '5.0', 'Intern': '5.0', 'Security C...","{'Graduate Researcher': '$26,598 per year', 'R...",8,Govt_Services,7,4.0,3.7,3.6,3.7,4.0,36279.933333
4,4.1,501,"{'Work Happiness Score': '70', 'Learning': '81...",89,181,NaN,"{'Program Coordinator': '$48,016 per year', 'R...",8,Govt_Services,7,4.1,3.8,3.7,3.7,3.9,45855.000000
5,4.0,183,NaN,76,84,"{'Laboratory Assistant': '5.0', 'Senior Resear...",NaN,8,Commodities,9,4.0,3.7,3.4,3.6,3.9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6472,3.6,174,NaN,65,63,NaN,"{'Registered Nurse': '$55,140 per year', 'Regi...",7,Healthcare_NGO,5,3.5,3.4,3.1,3.0,3.3,53226.800000
6473,2.5,255,"{'Work Happiness Score': '46', 'Achievement': ...",44,121,"{'Vehicle Operator': '4.0', 'Inspector': '3.9'...","{'Shuttle Driver': '$12.54 per hour', 'Driver'...",7,Govt_Services,5,2.4,2.6,2.4,2.2,2.4,28336.533333
6474,3.9,1600,"{'Work Happiness Score': '72', 'Achievement': ...",87,485,"{'Office Administrator': '5.0', 'Senior Field ...","{'Logistics Analyst': '$62,419 per year', 'Sys...",8,Financials,7,3.8,3.9,3.4,3.5,3.7,83891.000000
6475,3.3,311,NaN,58,131,"{'Recruiter': '5.0', 'Office Assistant': '4.8'...","{'Food Service Associate': '$12.14 per hour', ...",8,Govt_Services,5,3.2,3.3,3.3,3.3,3.3,31330.200000


## Drop rows without happiness data

Write to excel and pickle

In [22]:
data_prepare1["happiness"] = data_prepare1['happiness'].replace("Management", 'Happiness_Management', regex=True)

data_prepare2 = data_prepare1.dropna(subset = ['happiness'])
df = data_prepare2
df["happiness"] = df["happiness"].apply(lambda x : dict(eval(x)))
df1 = df["happiness"].apply(pd.Series)

data_prepare2 = pd.concat([df, df1], axis=1).drop('happiness', axis=1)

data_prepare2['Work Happiness Score'] = pd.to_numeric(data_prepare2['Work Happiness Score'])
data_prepare2.info()

/var/folders/xr/3fgdpnbj38v_32gf1_x3h2x00000gn/T/ipykernel_40533/3696752805.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["happiness"] = df["happiness"].apply(lambda x : dict(eval(x)))


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3117 entries, 3 to 6474
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   rating                    3117 non-null   float64
 1   reviews                   3117 non-null   int64  
 2   ceo_approval              3117 non-null   int64  
 3   ceo_count                 3117 non-null   int64  
 4   roles                     2753 non-null   object 
 5   salary                    2969 non-null   object 
 6   employees                 3117 non-null   int64  
 7   industry                  3117 non-null   object 
 8   revenue                   3117 non-null   int64  
 9   Work/Life Balance         3117 non-null   float64
 10  Compensation/Benefits     3117 non-null   float64
 11  Job Security/Advancement  3117 non-null   float64
 12  Management                3117 non-null   float64
 13  Culture                   3117 non-null   float64
 14  avg_sala

In [23]:
data_prepare2 = data_prepare2.drop(['Appreciation', 'Purpose','Learning','Support','Achievement', 'Flexibility',
       'Trust','Energy','Inclusion','Belonging', 'Happiness_Management', 'Compensation'], axis=1)
data_prepare2 = data_prepare2.reset_index(drop=True)
data_prepare2.index = np.arange(1, len(data_prepare2) + 1)
data_prepare2.head()

,rating,reviews,ceo_approval,ceo_count,roles,salary,employees,industry,revenue,Work/Life Balance,Compensation/Benefits,Job Security/Advancement,Management,Culture,avg_salary,Work Happiness Score
1,4.2,803,83,275,"{'Mentor': '5.0', 'Intern': '5.0', 'Security C...","{'Graduate Researcher': '$26,598 per year', 'R...",8,Govt_Services,7,4.0,3.7,3.6,3.7,4.0,36279.933333,76
2,4.1,501,89,181,NaN,"{'Program Coordinator': '$48,016 per year', 'R...",8,Govt_Services,7,4.1,3.8,3.7,3.7,3.9,45855.000000,70
3,3.9,167,90,79,"{'Director of Human Resources': '5.0', 'Food R...","{'Housekeeper': '$12.57 per hour', 'Attendant'...",8,Leisure,5,3.6,3.7,3.5,3.4,3.8,25590.933333,71
4,2.7,541,41,233,"{'Technician': '3.8', 'Personal Technology Exp...","{'Service Technician': '$10.49 per hour', 'Des...",7,Tech,4,2.8,2.5,2.2,2.5,2.6,22207.466667,45
5,3.2,271,58,117,"{'Bookkeeper': '4.3', 'Delivery Specialist': '...","{'Yard Worker': '$8.14 per hour', 'Yard Specia...",7,Real_Estate,7,3.2,3.0,2.6,2.8,3.0,23400.000000,60


## Export processed data to excel and pickle (V2)

In [24]:
data_prepare2.to_excel("../dataset/data_prepare2.xlsx")
data_prepare2.to_pickle('../dataset/data_prepare2.pickle')